In [1]:
import os
import numpy as np
import nibabel as nib
import Config
# import time
import datetime
from scipy.ndimage import gaussian_filter
from scipy.ndimage import uniform_filter
# from scipy.ndimage import sobel
from skimage.measure import block_reduce
# from skimage.transform import resize

# from skimage import exposure

# import matplotlib
# import matplotlib.pyplot as plt

import Multi_Threads
# from threading import Thread

# matplotlib.use('TkAgg')

branch_name = 'ROI_2J'

num_threads = 16  # 5 * 67 = 335

data_path_id = Config.Brats_Path_ID()
training_path = data_path_id.path_training_cases
validation_path = data_path_id.path_validation_cases
brats_training_ID = data_path_id.training_cases
brats_validation_ID = data_path_id.validation_cases

# total_patient = len(brats_training_ID)

img_H = 240
img_W = 240
img_D = 155

print('Start testing: ')


def count_pixel(arr, target):
    mask = (arr == target)
    arr_new = arr[mask]
    return arr_new.size


def normalization(data):
    _range = np.max(data) - np.min(data)
    if _range == 0:
        return_data = data - np.min(data)
    else:
        return_data = (data - np.min(data)) / _range
    return return_data


def normalization_signal(data):
    mask_data = data.copy()
    mask_data[mask_data != 0] = 1

    _range = np.max(data[data != 0]) - np.min(data[data != 0])
    if _range == 0:
        return_data = data - np.min(data[data != 0])
    else:
        return_data = (data - np.min(data[data != 0])) / _range
    return return_data * mask_data


def standardization(data):
    mu = np.mean(data)
    sigma = np.std(data)
    if sigma == 0:
        return_data = (data - mu)
    else:
        return_data = (data - mu) / sigma
    return return_data


def standardize_signal(data):
    mean_data = np.mean(data[data != 0])
    std_data = np.std(data[data != 0])

    if std_data == 0:
        standard_data = data - mean_data
    else:
        standard_data = (data - mean_data) / std_data

    return standard_data


def gaussian_lowpass3D(input_data, sigma, kernel_size):
    return gaussian_filter(input_data, sigma=sigma, mode='mirror', truncate=((kernel_size - 1) / 2 - 0.5) / sigma)  # kernel size = 2 * int(truncate * sd + 0.5) + 1


def generate_modulation_function(hist1, hist2, clip_value1, clip_value2):
    size_hist = np.shape(hist1)

    hist1[:, 0] = 0
    hist2[:, 0] = 0

    if np.sum(hist1[:, 22:size_hist[1]]) <= np.sum(hist2[:, 22:size_hist[1]]):
        hist_select = np.copy(hist1)
    else:
        hist_select = np.copy(hist2)

    hist_modulation = np.sum(hist_select, axis=0)

    hist_modulation[0] = hist_modulation[1]
    hist_modulation = uniform_filter(hist_modulation.astype(np.float32), size=7, mode='mirror')
    hist_modulation = normalization(hist_modulation)

    for i in range(size_hist[1]):
        if hist_modulation[i] == 1:
            max_bin = i
            break

    for i in np.arange(max_bin, size_hist[1], 1):
        if hist_modulation[i] <= clip_value1:
            clip_bin1 = i
            break

    for i in np.arange(clip_bin1, size_hist[1], 1):
        if hist_modulation[i] <= clip_value2:
            clip_bin2 = i
            break

    hist_modulation[0:clip_bin1] = hist_modulation[clip_bin1]
    hist_modulation[clip_bin2:size_hist[1]] = hist_modulation[clip_bin2]

    hist_modulation = uniform_filter(hist_modulation.astype(np.float32), size=5, mode='mirror')
    hist_modulation = 1 / hist_modulation
    hist_modulation = uniform_filter(hist_modulation.astype(np.float32), size=5, mode='mirror')
    hist_modulation = normalization(hist_modulation)

    '''
    plt.figure(1)
    plt.plot(hist_modulation)
    plt.show()
    '''

    return hist_modulation


def difference_hist_v9(hist1, hist2, modulation_function_original, gamma, beata, mode=3):
    size_hist = np.shape(hist1)

    modulation_function = modulation_function_original ** gamma + beata
    # modulation_function[0] = 0
    # modulation_function = normalization(modulation_function)

    hist1[:, 0] = 0

    if mode == 1 or hist2 is None:
        diff_hist_2d = np.multiply(hist1, modulation_function)  # np.multiply(half_histogram_clip(hist1), tumoral_asymmetry_f)

    else:
        hist2[:, 0] = 0
        diff_hist_2d = np.multiply(np.abs(hist1 - hist2), modulation_function)  # np.divide(np.abs(hist1 - hist2), hist_norm)

    '''
    bin_mean = np.arange(0.0, 1, 0.025) + 0.025 / 2
    gray_2d = np.zeros(size_hist, dtype=np.float32)
    for i in range(size_hist[0]):
        gray_2d[i, :] = diff_hist_2d[i, :] * bin_mean

    mean_profile = np.zeros(size_hist[0], dtype=np.float32)
    std_profile = np.zeros(size_hist[0], dtype=np.float32)

    for i in range(size_hist[0]):
        if np.sum(gray_2d[i, :]) != 0 and np.sum(diff_hist_2d[i, :]) != 0:
            mean_profile[i] = np.sum(gray_2d[i, :]) / np.sum(diff_hist_2d[i, :])
            for j in range(size_hist[1]):
                std_profile[i] = std_profile[i] + diff_hist_2d[i, j] * (bin_mean[j] - mean_profile[i]) ** 2
            std_profile[i] = np.sqrt(std_profile[i] / np.sum(diff_hist_2d[i, :]))

    # asymmetry_profile = np.sum(diff_hist_2d, 1) * (mean_profile + np.exp(-0.5/(std_profile+0.000001)))
    mean_profile = uniform_filter(mean_profile, size=7, mode='constant')

    tumoral_asymmetry = np.copy(diff_hist_2d)
    for i in range(size_hist[0]):
        tumoral_asymmetry[i, :] = diff_hist_2d[i, :] * mean_profile[i]
    '''

    tumoral_asymmetry = normalization(diff_hist_2d)
    # tumoral_asymmetry = tumoral_asymmetry - 0.00    # offset
    # tumoral_asymmetry[tumoral_asymmetry < 0] = 0

    asymmetry_profile = np.sum(tumoral_asymmetry, 1)  # np.sum(diff_hist_2d, 1) * mean_profile
    asymmetry_profile = normalization(asymmetry_profile)

    return asymmetry_profile, tumoral_asymmetry


def removal_non_interest_slices_v9(asymmetry_profile, mode=3):
    size_hist = np.shape(asymmetry_profile)
    size_hist = size_hist[0]

    temp_interest = np.ones(size_hist, dtype=np.float32)

    # filter
    asymmetry_profile_filter = np.zeros(size_hist, dtype=np.float32)
    for i in np.arange(5, size_hist-5, 1):
        if asymmetry_profile[i] != 0 or (
                np.sum(asymmetry_profile[i - 5:i]) != 0 and np.sum(asymmetry_profile[i + 1:i + 6]) != 0):
            asymmetry_profile_filter[i] = np.mean(asymmetry_profile[i - 5:i + 6])
    # normalized
    asymmetry_profile_filter = asymmetry_profile_filter / np.max(asymmetry_profile_filter)

    # removal
    temp_sort = np.sort(asymmetry_profile_filter)
    temp_sort = temp_sort[temp_sort != 0]

    size_sort = np.shape(temp_sort)
    size_sort = size_sort[0]

    '''
    peak_value = np.min([np.mean(temp_sort[int(np.rint(0.6 * size_sort + 0.001) - 1):size_sort]), 0.45])
    minima_threshold = np.min([np.max([np.mean(temp_sort[0:int(np.rint(0.6 * size_sort + 0.001))]), 0.05]), 0.2])
    bottom_value = np.max([np.mean(temp_sort[0:1 + int(np.rint(0.02 * size_sort + 0.001))]), 0.01])
    threshold_value = np.max([np.mean(temp_sort[0:1 + int(np.rint(0.2 * size_sort + 0.001))]), 0.02])
    '''

    '''
    peak_value = 0.5
    minima_threshold = 0.25
    bottom_value = 0.04
    threshold_value = 0.08
    '''
    peak_value = 0.6
    minima_threshold = 0.25
    bottom_value = 0.06
    threshold_value = 0.1

    peak_start = 0
    peak_end = size_hist - 1
    for i in range(size_hist):
        if asymmetry_profile_filter[i] >= peak_value:
            peak_start = i
            break
    for i in np.arange(size_hist-1, -1, -1):
        if asymmetry_profile_filter[i] >= peak_value:
            peak_end = i
            break

    interest_start = 0
    interest_end = size_hist - 1

    for i in np.arange(peak_start, 2, -1):
        if asymmetry_profile_filter[i] <= minima_threshold and asymmetry_profile_filter[i] <= asymmetry_profile_filter[i - 1] and \
                asymmetry_profile_filter[i] <= asymmetry_profile_filter[i - 2] and asymmetry_profile_filter[i] <= asymmetry_profile_filter[i + 1]:
            interest_start = i
            break

    for i in np.arange(peak_end, size_hist - 3, 1):
        if asymmetry_profile_filter[i] <= minima_threshold and asymmetry_profile_filter[i] <= asymmetry_profile_filter[i + 1] and \
                asymmetry_profile_filter[i] <= asymmetry_profile_filter[i + 2] and asymmetry_profile_filter[i] <= asymmetry_profile_filter[i - 1]:
            interest_end = i
            break

    temp_interest[0:interest_start] = 0
    temp_interest[interest_end:size_hist] = 0

    if mode == 3:
        for i in range(125):
            if asymmetry_profile_filter[i] < threshold_value:
                temp_interest[i] = 0
        for i in np.arange(125, size_hist, 1):
            if asymmetry_profile_filter[i] < bottom_value:
                temp_interest[i] = 0
    elif mode == 2:
        for i in range(size_hist):
            if asymmetry_profile_filter[i] < threshold_value:  # !!!!! threshold_value <-- bottom_value
                temp_interest[i] = 0
    elif mode == 1:
        for i in range(size_hist):
            if asymmetry_profile_filter[i] < threshold_value:  # !!!!! threshold_value <-- bottom_value
                temp_interest[i] = 0

    # refinement
    for i in np.arange(3, size_hist - 3, 1):
        if temp_interest[i] == 0 and (np.sum(temp_interest[i - 3:i + 4]) >= 5):
            temp_interest[i] = 1
    for i in np.arange(1, size_hist - 7, 1):
        if temp_interest[i] == 1 and temp_interest[i - 1] == 0 and (np.sum(temp_interest[i:i + 7]) < 7):
            temp_interest[i] = 0

    return temp_interest, asymmetry_profile_filter


def WT_detection_v6(input_img, interest_slice_3rd, interest_slice_2nd, interest_slice_1st, number_voxel_cube, threshold_pred=None):
    # mask_cube = np.copy(input_img)
    # mask_cube[mask_cube != 0] = 1

    # soi_cube = uniform_filter(soi_cube, size=3, mode='mirror')
    # soi_cube = soi_cube * mask_cube
    # soi_cube = normalization_signal(soi_cube)

    #################### extend cuboid
    extend_num = 5

    for i in range(155):
        if interest_slice_3rd[i] > 0:
            start_3rd = np.max([i - extend_num, 0])
            break

    for i in np.arange(154, -1, -1, dtype=np.int16):
        if interest_slice_3rd[i] > 0:
            end_3rd = np.min([i + extend_num, 154])
            break

    for i in range(240):
        if interest_slice_2nd[i] > 0:
            start_2nd = np.max([i - extend_num, 0])
            break

    for i in np.arange(239, -1, -1, dtype=np.int16):
        if interest_slice_2nd[i] > 0:
            end_2nd = np.min([i + extend_num, 239])
            break

    for i in range(240):
        if interest_slice_1st[i] > 0:
            start_1st = np.max([i - extend_num, 0])
            break

    for i in np.arange(239, -1, -1, dtype=np.int16):
        if interest_slice_1st[i] > 0:
            end_1st = np.min([i + extend_num, 239])
            break

    crop_cuboid = np.zeros((240, 240, 155), dtype=np.float32)
    crop_cuboid[start_1st:end_1st, start_2nd:end_2nd, start_3rd:end_3rd] = input_img[start_1st:end_1st, start_2nd:end_2nd, start_3rd:end_3rd]

    ######################################

    if threshold_pred is None:
        if np.max(crop_cuboid) > 0:
            threshold_pred = np.mean(crop_cuboid[crop_cuboid > 0])
        else:
            threshold_pred = 0


    crop_cuboid[crop_cuboid < threshold_pred] = 0
    crop_cuboid[crop_cuboid != 0] = 1

    out_binary_img = uniform_filter(crop_cuboid, size=5, mode='mirror')

    # threshold_binary = 0.15
    '''
    threshold_binary = 0.002 * (number_voxel_cube / 10000.0) + 0.1  # threshold_binary_rs
    threshold_binary = np.min([threshold_binary, 0.4])
    threshold_binary = np.max([threshold_binary, 0.05])
    '''
    threshold_binary = 0.25

    out_binary_img[out_binary_img < threshold_binary] = 0
    out_binary_img[out_binary_img != 0] = 1

    return out_binary_img  # * mask_cube


def compare_similarity(pred_in, true_in):
    pred_data = normalization(pred_in)
    true_data = normalization(true_in)

    covariance = np.mean((pred_data - np.mean(pred_data)) * (true_data - np.mean(true_data)))
    correlation = covariance / np.std(pred_data) / np.std(true_data)
    mse = np.mean((pred_data - true_data) ** 2)

    c1 = (0.01 * 1) ** 2
    c2 = (0.03 * 1) ** 2
    c3 = c2 / 2
    ssim1 = (2 * np.mean(pred_data) * np.mean(true_data) + c1) / ((np.mean(pred_data)) ** 2 + (np.mean(true_data)) ** 2 + c1)
    ssim2 = (2 * np.std(pred_data) * np.std(true_data) + c2) / (np.var(pred_data) + np.var(true_data) + c2)
    ssim3 = (covariance + c3) / (np.std(pred_data) * np.std(true_data) + c3)
    ssim = ssim1 * ssim2 * ssim3

    return covariance, correlation, mse, ssim


def detect_non_interest_slices_symmetry_v2(img_data, seg_soi=None, patient_path=None):
    size_img = np.shape(img_data)
    temp_img = img_data.copy()

    mask_img = img_data.copy()
    mask_img[mask_img != 0] = 1

    temp_img = uniform_filter(temp_img, size=5, mode='mirror')
    temp_img = temp_img * mask_img
    mean_img = np.mean(temp_img[temp_img != 0])
    temp_img[temp_img < mean_img] = 0

    temp_img = normalization_signal(temp_img)

    temp_img_morphology = np.copy(temp_img)

    temp_img_gt = np.copy(temp_img)
    temp_img_pred_dist_out = np.copy(temp_img)

    ################################################################################

    output_path_histogram_soi = './Histogram_SOI/'
    ################################################################################################
    if (seg_soi is not None) and (patient_path is not None):
        if not os.path.exists(output_path_histogram_soi):
            try:
                os.makedirs(output_path_histogram_soi)
            except:
                folder_exist = True

        if patient_path[3] == '/':
            patient_path2 = patient_path[4:len(patient_path)]
        else:
            patient_path2 = patient_path

        temp_seg_soi = np.copy(seg_soi)
        temp_seg_soi[temp_seg_soi != 0] = 1

        # ground truth line 3rd
        ground_truth_line_3rd = np.zeros(size_img[2], dtype=np.float32)
        for j in range(size_img[2]):
            ground_truth_line_3rd[j] = np.max(temp_seg_soi[:, :, j])
        ground_truth_line_3rd[ground_truth_line_3rd > 0] = 1

        # ground truth line 2nd
        ground_truth_line_2nd = np.zeros(size_img[1], dtype=np.float32)
        for j in range(size_img[1]):
            ground_truth_line_2nd[j] = np.max(temp_seg_soi[:, j, :])
        ground_truth_line_2nd[ground_truth_line_2nd > 0] = 1

        # ground truth line 1st
        ground_truth_line_1st = np.zeros(size_img[0], dtype=np.float32)
        for j in range(size_img[0]):
            ground_truth_line_1st[j] = np.max(temp_seg_soi[j, :, :])
        ground_truth_line_1st[ground_truth_line_1st > 0] = 1

        np.savetxt((output_path_histogram_soi + patient_path2 + '_ground_truth_line_3rd.txt'),
                   ground_truth_line_3rd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_ground_truth_line_2nd.txt'),
                   ground_truth_line_2nd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_ground_truth_line_1st.txt'),
                   ground_truth_line_1st, fmt='%f')


    ######################################################################################
    bin_hist = np.arange(0.0, 1.025, 0.025)
    length_hist = np.shape(bin_hist)
    length_hist = length_hist[0] - 1


    ##################################
    if (seg_soi is not None) and (patient_path is not None):
        temp_seg = np.copy(seg_soi)
        temp_seg_copy = np.zeros(np.shape(temp_seg), dtype=np.float32)
        temp_seg_copy[:, :, :] = temp_seg[:, :, :]
        # get gt hist 3rd
        temp_seg = block_reduce(temp_seg_copy, block_size=(2, 2, 1), func=np.max)
        temp_seg[temp_seg > 0] = 1

        temp_img_cover = temp_seg * block_reduce(temp_img_gt, block_size=(2, 2, 1), func=np.mean)
        hist_global_gt_3rd = np.zeros((size_img[2], length_hist), dtype=np.float32)
        for i in range(size_img[2]):
            hist_global_gt_3rd[i, :], bin_ = np.histogram(temp_img_cover[:, :, i], bin_hist)
        hist_global_gt_3rd[:, 0] = 0

        # get gt hist 2nd
        temp_seg = block_reduce(temp_seg_copy, block_size=(2, 1, 2), func=np.max)
        temp_seg[temp_seg > 0] = 1

        temp_img_cover = temp_seg * block_reduce(temp_img_gt, block_size=(2, 1, 2), func=np.mean)
        hist_global_gt_2nd = np.zeros((size_img[1], length_hist), dtype=np.float32)
        for i in range(size_img[1]):
            hist_global_gt_2nd[i, :], bin_ = np.histogram(temp_img_cover[:, i, :], bin_hist)
        hist_global_gt_2nd[:, 0] = 0

        # get gt hist 1st
        temp_seg = block_reduce(temp_seg_copy, block_size=(1, 2, 2), func=np.max)
        temp_seg[temp_seg > 0] = 1

        temp_img_cover = temp_seg * block_reduce(temp_img_gt, block_size=(1, 2, 2), func=np.mean)
        hist_global_gt_1st = np.zeros((size_img[0], length_hist), dtype=np.float32)
        for i in range(size_img[0]):
            hist_global_gt_1st[i, :], bin_ = np.histogram(temp_img_cover[i, :, :], bin_hist)
        hist_global_gt_1st[:, 0] = 0

        np.savetxt((output_path_histogram_soi + patient_path2 + '_hist_global_gt_3rd.txt'),
                   hist_global_gt_3rd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_hist_global_gt_2nd.txt'),
                   hist_global_gt_2nd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_hist_global_gt_1st.txt'),
                   hist_global_gt_1st, fmt='%f')
        #################################

    ###########################################################################################
    # 3rd
    temp_img_3rd = np.zeros(np.shape(temp_img), dtype=np.float32)
    temp_img_3rd[:, :, :] = temp_img[:, :, :]
    temp_img_3rd = block_reduce(temp_img_3rd, block_size=(2, 2, 1), func=np.mean)

    size_img_3rd = np.shape(temp_img_3rd)
    img_up_3rd = temp_img_3rd[0:int(size_img_3rd[0] / 2), :, :]
    img_down_3rd = temp_img_3rd[int(size_img_3rd[0] / 2):size_img_3rd[0], :, :]
    img_down_3rd = np.flip(img_down_3rd, axis=0)

    hist_slice_up_3rd = np.zeros((size_img_3rd[2], length_hist), dtype=np.float32)
    hist_slice_down_3rd = np.zeros((size_img_3rd[2], length_hist), dtype=np.float32)

    for i in range(size_img_3rd[2]):
        hist_slice_up_3rd[i, :], bin_ = np.histogram(img_up_3rd[:, :, i], bin_hist)
        hist_slice_down_3rd[i, :], bin_ = np.histogram(img_down_3rd[:, :, i], bin_hist)

    hist_slice_up_3rd[:, 0] = 0
    hist_slice_down_3rd[:, 0] = 0
    hist_slice_up_3rd = uniform_filter(hist_slice_up_3rd, size=3, mode='mirror')
    hist_slice_down_3rd = uniform_filter(hist_slice_down_3rd, size=3, mode='mirror')

    modulation_function_original = generate_modulation_function(hist_slice_up_3rd, hist_slice_down_3rd, 0.5, 0.05)

    asymmetry_profile_3rd, tumoral_asymmetry_3rd = difference_hist_v9(hist_slice_up_3rd, hist_slice_down_3rd, modulation_function_original, 1.8, 0.02, mode=3)
    interest_slice_3rd, asymmetry_profile_filter_3rd = removal_non_interest_slices_v9(asymmetry_profile_3rd, mode=3)

    if (seg_soi is not None) and (patient_path is not None):
        np.savetxt((output_path_histogram_soi + patient_path2 + '_tumoral_asymmetry_3rd.txt'),
                   tumoral_asymmetry_3rd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_asymmetry_profile_filter_3rd.txt'),
                   asymmetry_profile_filter_3rd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_interest_slice_3rd.txt'),
                   interest_slice_3rd, fmt='%f')

        np.savetxt((output_path_histogram_soi + patient_path2 + '_hist_slice_up_3rd.txt'),
                   hist_slice_up_3rd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_hist_slice_down_3rd.txt'),
                   hist_slice_down_3rd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_modulation_function_original.txt'),
                   modulation_function_original, fmt='%f')


    ###########################################################################################################
    # 2nd
    tumoral_asymmetry_3rd_remove_noninterest = np.copy(tumoral_asymmetry_3rd)

    for i in range(size_img[2]):
        if interest_slice_3rd[i] == 0:
            temp_img[:, :, i] = 0
            tumoral_asymmetry_3rd_remove_noninterest[i, :] = 0

    # temp_img = normalization_signal(temp_img)

    temp_img_2nd = np.zeros(np.shape(temp_img), dtype=np.float32)
    temp_img_2nd[:, :, :] = temp_img[:, :, :]
    temp_img_2nd = block_reduce(temp_img_2nd, block_size=(2, 1, 2), func=np.mean)

    size_img_2nd = np.shape(temp_img_2nd)
    img_up_2nd = temp_img_2nd[0:int(size_img_2nd[0] / 2), :, :]
    img_down_2nd = temp_img_2nd[int(size_img_2nd[0] / 2):size_img_2nd[0], :, :]
    img_down_2nd = np.flip(img_down_2nd, axis=0)

    hist_slice_up_2nd = np.zeros((size_img_2nd[1], length_hist), dtype=np.float32)
    hist_slice_down_2nd = np.zeros((size_img_2nd[1], length_hist), dtype=np.float32)

    for i in range(size_img_2nd[1]):
        hist_slice_up_2nd[i, :], bin_ = np.histogram(img_up_2nd[:, i, :], bin_hist)
        hist_slice_down_2nd[i, :], bin_ = np.histogram(img_down_2nd[:, i, :], bin_hist)

    hist_slice_up_2nd[:, 0] = 0
    hist_slice_down_2nd[:, 0] = 0
    hist_slice_up_2nd = uniform_filter(hist_slice_up_2nd, size=3, mode='mirror')
    hist_slice_down_2nd = uniform_filter(hist_slice_down_2nd, size=3, mode='mirror')

    asymmetry_profile_2nd, tumoral_asymmetry_2nd = difference_hist_v9(hist_slice_up_2nd, hist_slice_down_2nd, modulation_function_original, 1.8, 0.02, mode=2)
    interest_slice_2nd, asymmetry_profile_filter_2nd = removal_non_interest_slices_v9(asymmetry_profile_2nd, mode=2)

    if (seg_soi is not None) and (patient_path is not None):
        np.savetxt((output_path_histogram_soi + patient_path2 + '_tumoral_asymmetry_2nd.txt'),
                   tumoral_asymmetry_2nd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_asymmetry_profile_filter_2nd.txt'),
                   asymmetry_profile_filter_2nd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_interest_slice_2nd.txt'),
                   interest_slice_2nd, fmt='%f')

    ###########################################################################################################
    # 1st
    tumoral_asymmetry_2nd_remove_noninterest = np.copy(tumoral_asymmetry_2nd)

    for i in range(size_img[1]):
        if interest_slice_2nd[i] == 0:
            temp_img[:, i, :] = 0
            tumoral_asymmetry_2nd_remove_noninterest[i, :] = 0

    # temp_img = normalization_signal(temp_img)

    temp_img_1st = np.zeros(np.shape(temp_img), dtype=np.float32)
    temp_img_1st[:, :, :] = temp_img[:, :, :]
    temp_img_1st = block_reduce(temp_img_1st, block_size=(1, 2, 2), func=np.mean)

    size_img_1st = np.shape(temp_img_1st)

    hist_slice_1st = np.zeros((size_img_1st[0], length_hist), dtype=np.float32)

    for i in range(size_img_1st[0]):
        hist_slice_1st[i, :], bin_ = np.histogram(temp_img_1st[i, :, :], bin_hist)

    hist_slice_1st[:, 0] = 0
    hist_slice_1st = uniform_filter(hist_slice_1st, size=3, mode='mirror')

    asymmetry_profile_1st, tumoral_asymmetry_1st = difference_hist_v9(hist_slice_1st, None, modulation_function_original, 1.8, 0.02, mode=1)
    interest_slice_1st, asymmetry_profile_filter_1st = removal_non_interest_slices_v9(asymmetry_profile_1st, mode=1)

    if (seg_soi is not None) and (patient_path is not None):
        np.savetxt((output_path_histogram_soi + patient_path2 + '_tumoral_asymmetry_1st.txt'),
                   tumoral_asymmetry_1st, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_asymmetry_profile_filter_1st.txt'),
                   asymmetry_profile_filter_1st, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_interest_slice_1st.txt'),
                   interest_slice_1st, fmt='%f')

    ########################################################
    # predict distribution
    ########################################################

    tumoral_asymmetry_1st_remove_noninterest = np.copy(tumoral_asymmetry_1st)

    for i in range(size_img[0]):
        if interest_slice_1st[i] == 0:
            temp_img[i, :, :] = 0
            tumoral_asymmetry_1st_remove_noninterest[i, :] = 0


    # temp_img = normalization_signal(temp_img)

    output_binary_cuboid = np.copy(temp_img)
    output_binary_cuboid[output_binary_cuboid != 0] = 1

    '''
    temp_img_pred_dist = np.zeros(np.shape(temp_img), dtype=np.float32)
    temp_img_pred_dist[:, :, :] = temp_img[:, :, :]
    temp_img_pred_dist = block_reduce(temp_img_pred_dist, block_size=(1, 2, 2), func=np.mean)

    size_img_pred_dist = np.shape(temp_img_pred_dist)

    hist_slice_pred_dist = np.zeros((size_img_pred_dist[0], length_hist), dtype=np.float32)

    for i in range(size_img_pred_dist[0]):
        hist_slice_pred_dist[i, :], bin_ = np.histogram(temp_img_pred_dist[i, :, :], bin_hist)

    hist_slice_pred_dist[:, 0] = 0
    hist_slice_pred_dist = uniform_filter(hist_slice_pred_dist, size=3, mode='mirror')

    _, tumoral_asymmetry_pred_dist = difference_hist_predict_distribution(hist_slice_pred_dist, None, tumoral_asymmetry=tumoral_asymmetry_1st_remove_noninterest, mode=1)

    '''

    # predict 1st
    temp_img_pred_dist_1st = np.zeros(np.shape(temp_img), dtype=np.float32)
    temp_img_pred_dist_1st[:, :, :] = temp_img[:, :, :]
    temp_img_pred_dist_1st = block_reduce(temp_img_pred_dist_1st, block_size=(1, 2, 2), func=np.mean)

    size_img_pred_dist_1st = np.shape(temp_img_pred_dist_1st)

    hist_slice_pred_dist_1st = np.zeros((size_img_pred_dist_1st[0], length_hist), dtype=np.float32)

    for i in range(size_img_pred_dist_1st[0]):
        hist_slice_pred_dist_1st[i, :], bin_ = np.histogram(temp_img_pred_dist_1st[i, :, :], bin_hist)

    hist_slice_pred_dist_1st[:, 0] = 0
    # hist_slice_pred_dist_1st = uniform_filter(hist_slice_pred_dist_1st, size=3, mode='mirror')

    modulation_function_original_pred = generate_modulation_function(hist_slice_up_3rd, hist_slice_down_3rd, 0.6, 0.15)
    _, tumoral_asymmetry_pred_dist_1st = difference_hist_v9(hist_slice_pred_dist_1st, None, modulation_function_original_pred, 1.5, 1.0/8.0, mode=1)


    # tumoral_asymmetry_pred_dist_1st = tumoral_asymmetry_1st_remove_noninterest

    cdf_pred_dist = np.sum(tumoral_asymmetry_pred_dist_1st, axis=0)
    cdf_pred_dist[0] = 0
    cdf_pred_dist = np.cumsum(cdf_pred_dist)
    if np.max(cdf_pred_dist) != 0:
        cdf_pred_dist = cdf_pred_dist / np.max(cdf_pred_dist)

    hist_cube, _ = np.histogram(temp_img, bin_hist)
    hist_cube[0] = 0
    # cdf_cube = np.cumsum(hist_cube)
    # if np.max(cdf_cube) != 0:
    #     cdf_cube = cdf_cube / np.max(cdf_cube)

    number_voxel_cube = np.sum(hist_cube)

    if (seg_soi is not None) and (patient_path is not None):
        temp_gt_pred_dist_out = np.copy(seg_soi)
        temp_gt_pred_dist_out[temp_gt_pred_dist_out != 0] = 1
        hist_cube_gt = temp_img_pred_dist_out * temp_gt_pred_dist_out
        hist_cube_gt, _ = np.histogram(hist_cube_gt, bin_hist)
        hist_cube_gt[0] = 0
        # cdf_gt = np.cumsum(cdf_gt)
        # if np.max(cdf_gt) != 0:
        #     cdf_gt = cdf_gt / np.max(cdf_gt)

        np.savetxt((output_path_histogram_soi + patient_path2 + '_tumoral_asymmetry_pred_dist_1st.txt'),
                   tumoral_asymmetry_pred_dist_1st, fmt='%f')
        # np.savetxt((output_path_histogram_soi + patient_path2 + '_cdf_pred_dist.txt'), cdf_pred_dist, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_hist_cube.txt'),
                   hist_cube, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_hist_cube_gt.txt'),
                   hist_cube_gt, fmt='%f')

        np.savetxt((output_path_histogram_soi + patient_path2 + '_modulation_function_original_pred.txt'),
                   modulation_function_original_pred, fmt='%f')

        ########
        # predict 3rd
        temp_img_pred_dist_3rd = np.zeros(np.shape(temp_img), dtype=np.float32)
        temp_img_pred_dist_3rd[:, :, :] = temp_img[:, :, :]
        temp_img_pred_dist_3rd = block_reduce(temp_img_pred_dist_3rd, block_size=(2, 2, 1), func=np.mean)

        size_img_pred_dist_3rd = np.shape(temp_img_pred_dist_3rd)

        hist_slice_pred_dist_3rd = np.zeros((size_img_pred_dist_3rd[2], length_hist), dtype=np.float32)

        for i in range(size_img_pred_dist_3rd[2]):
            hist_slice_pred_dist_3rd[i, :], bin_ = np.histogram(temp_img_pred_dist_3rd[:, :, i], bin_hist)

        hist_slice_pred_dist_3rd[:, 0] = 0
        # hist_slice_pred_dist_3rd = uniform_filter(hist_slice_pred_dist_3rd, size=3, mode='mirror')

        _, tumoral_asymmetry_pred_dist_3rd = difference_hist_v9(hist_slice_pred_dist_3rd, None, modulation_function_original_pred, 1.5, 1.0/8.0, mode=1)

        ########
        # predict 2nd
        temp_img_pred_dist_2nd = np.zeros(np.shape(temp_img), dtype=np.float32)
        temp_img_pred_dist_2nd[:, :, :] = temp_img[:, :, :]
        temp_img_pred_dist_2nd = block_reduce(temp_img_pred_dist_2nd, block_size=(2, 1, 2), func=np.mean)

        size_img_pred_dist_2nd = np.shape(temp_img_pred_dist_2nd)

        hist_slice_pred_dist_2nd = np.zeros((size_img_pred_dist_2nd[1], length_hist), dtype=np.float32)

        for i in range(size_img_pred_dist_2nd[1]):
            hist_slice_pred_dist_2nd[i, :], bin_ = np.histogram(temp_img_pred_dist_2nd[:, i, :], bin_hist)

        hist_slice_pred_dist_2nd[:, 0] = 0
        # hist_slice_pred_dist_2nd = uniform_filter(hist_slice_pred_dist_2nd, size=3, mode='mirror')

        _, tumoral_asymmetry_pred_dist_2nd = difference_hist_v9(hist_slice_pred_dist_2nd, None, modulation_function_original_pred, 1.5, 1.0/8.0, mode=1)

        #################

        np.savetxt((output_path_histogram_soi + patient_path2 + '_tumoral_asymmetry_pred_dist_3rd.txt'),
                   tumoral_asymmetry_pred_dist_3rd, fmt='%f')
        np.savetxt((output_path_histogram_soi + patient_path2 + '_tumoral_asymmetry_pred_dist_2nd.txt'),
                   tumoral_asymmetry_pred_dist_2nd, fmt='%f')

        #### compare_similarity
        similarity_value = np.zeros((1, 12), dtype=np.float32)
        similarity_value[0, 0:4] = compare_similarity(tumoral_asymmetry_pred_dist_3rd, hist_global_gt_3rd)
        similarity_value[0, 4:8] = compare_similarity(tumoral_asymmetry_pred_dist_2nd, hist_global_gt_2nd)
        similarity_value[0, 8:12] = compare_similarity(tumoral_asymmetry_pred_dist_1st, hist_global_gt_1st)
    #################################################################################################
    # WT
    ###############################################################################################
    threshold_pred_gray_level = None

    for i in range(length_hist):
        if cdf_pred_dist[i] >= 20.0 / 100.0:
            threshold_pred_gray_level = i / length_hist
            break

    if threshold_pred_gray_level < 0.15 or threshold_pred_gray_level > 0.45:
        threshold_pred_gray_level = 0.35

    # threshold_pred_gray_level = np.max([0.2, threshold_pred_gray_level])
    # threshold_pred_gray_level = np.min([0.4, threshold_pred_gray_level])

    if np.max(hist_cube) > 19000:
        threshold_pred_gray_level = 0.4

    output_binary_wt = WT_detection_v6(temp_img_morphology, interest_slice_3rd, interest_slice_2nd, interest_slice_1st,
                                       number_voxel_cube, threshold_pred=threshold_pred_gray_level)

    if (seg_soi is not None) and (patient_path is not None):
        return output_binary_cuboid * mask_img, output_binary_wt * mask_img, interest_slice_3rd, interest_slice_2nd, interest_slice_1st, similarity_value
    else:
        return output_binary_cuboid * mask_img, output_binary_wt * mask_img, interest_slice_3rd, interest_slice_2nd, interest_slice_1st


def clip_histogram_equalization(input_img, num_bin=256, clip_limit=0.01):
    temp_img = np.copy(input_img)
    temp_mask = np.copy(input_img)
    temp_mask[temp_mask > 0] = 1

    hist, bin_edges = np.histogram(temp_img[temp_img > 0], bins=num_bin)
    bin_centers = (bin_edges[0:num_bin] + bin_edges[1:num_bin+1]) / 2.0

    hist = hist / np.sum(hist)

    '''
    plt.figure(1)
    plt.subplot(131)
    plt.plot(hist)
    plt.title('histogram')
    '''


    hist_excess = hist - clip_limit
    excess_sum = np.sum(hist_excess[hist_excess > 0])
    hist[hist > clip_limit] = clip_limit
    hist_clip = hist + excess_sum/num_bin

    img_cdf = hist_clip.cumsum()
    img_cdf = img_cdf / float(img_cdf[-1])

    '''
    plt.subplot(132)
    plt.plot(hist_clip)
    plt.title('clipped histogram')

    plt.subplot(133)
    plt.plot(img_cdf)
    plt.title('CDF')
    plt.show()
    '''

    out = np.interp(temp_img.flat, bin_centers, img_cdf)
    out = out.reshape(temp_img.shape)

    out = normalization_signal(out * temp_mask)

    return out.astype(np.float32)


def intensity_enhancing_v3(input_img, num_bin=256):
    size_img = np.shape(input_img)
    temp_img = np.copy(input_img)
    temp_img = temp_img.astype(np.float32)

    temp_mask = np.copy(temp_img)
    temp_mask[temp_mask > 0] = 1

    temp_img = uniform_filter(temp_img, size=3, mode='mirror')
    temp_img = temp_img * temp_mask
    temp_img = normalization_signal(temp_img)

    # mean_img = np.mean(temp_img[temp_img != 0])
    median_img = np.median(temp_img[temp_img != 0])

    # print('mean, median', mean_img, median_img)

    if median_img < 0.1:
        clip_limit = 0.01
        is_low_contrast = 'VeryLow'
    elif median_img < 0.2 and median_img >= 0.1:
        clip_limit = 0.008
        is_low_contrast = 'Low'
    else:
        clip_limit = 0.005
        is_low_contrast = 'Fine'

    threshold = median_img

    # print('mean, median: ', mean_img, median_img)

    temp_img_low_gray = np.copy(temp_img)
    temp_img_low_gray[temp_img_low_gray >= threshold] = 0

    temp_img_high_gray = np.copy(temp_img)
    temp_img_high_gray[temp_img < threshold] = 0
    temp_img_high_gray = normalization_signal(temp_img_high_gray)
    temp_img_high_gray_mask = np.copy(temp_img_high_gray)
    temp_img_high_gray_mask[temp_img_high_gray_mask > 0] = 1

    temp_img_high_gray = clip_histogram_equalization(temp_img_high_gray, num_bin=num_bin, clip_limit=clip_limit)

    temp_img_low_gray = normalization_signal(temp_img_low_gray) * threshold / 2.0
    temp_img_high_gray = normalization_signal(temp_img_high_gray) * (1 - threshold / 2.0) + threshold / 2.0
    temp_img_high_gray = temp_img_high_gray * temp_img_high_gray_mask

    out = uniform_filter(temp_img_low_gray + temp_img_high_gray, size=3, mode='mirror')
    out = out * temp_mask
    out = normalization_signal(out)

    return out.astype(np.float32), is_low_contrast


####################
# test post
#################

def Testing_Cases_in_ValidationSet(input_dir_path, patient_ID):
    path_flair = os.path.join(input_dir_path + patient_ID + '/' + patient_ID + '_flair.nii.gz')
    # path_t1 = os.path.join(input_dir_path + patient_ID + '/' + patient_ID + '_t1.nii.gz')
    # path_t1ce = os.path.join(input_dir_path + patient_ID + '/' + patient_ID + '_t1ce.nii.gz')
    # path_t2 = os.path.join(input_dir_path + patient_ID + '/' + patient_ID + '_t2.nii.gz')

    flair_temp = nib.load(path_flair)
    flair_temp_arr = flair_temp.get_fdata()
    flair_temp_arr_sq = np.squeeze(flair_temp_arr)

    '''
    flair_soi_cube = np.copy(flair_temp_arr_sq)
    flair_out = np.copy(flair_temp_arr_sq)

    t1_temp = nib.load(path_t1)
    t1_temp_arr = t1_temp.get_fdata()
    t1_temp_arr_sq = np.squeeze(t1_temp_arr)

    t1ce_temp = nib.load(path_t1ce)
    t1ce_temp_arr = t1ce_temp.get_fdata()
    t1ce_temp_arr_sq = np.squeeze(t1ce_temp_arr)

    t2_temp = nib.load(path_t2)
    t2_temp_arr = t2_temp.get_fdata()
    t2_temp_arr_sq = np.squeeze(t2_temp_arr)
    '''

    affine_array = flair_temp.affine

    ########################################################################################################
    flair_temp_arr_sq_enhance, is_low_contrast = intensity_enhancing_v3(flair_temp_arr_sq, num_bin=256)

    #################################################################################################
    '''
    interest_slice_remove_margin_3rd, interest_slice_remove_margin_2nd, interest_slice_remove_margin_1st = remove_margin(
        flair_temp_arr_sq, t1_temp_arr_sq, t1ce_temp_arr_sq, t2_temp_arr_sq)

    flair_remove_margin = np.copy(flair_temp_arr_sq_enhance)

    for j in range(img_D):
        if interest_slice_remove_margin_3rd[j] == 0:
            flair_remove_margin[:, :, j] = 0

    for j in range(img_W):
        if interest_slice_remove_margin_2nd[j] == 0:
            flair_remove_margin[:, j, :] = 0

    for j in range(img_H):
        if interest_slice_remove_margin_1st[j] == 0:
            flair_remove_margin[j, :, :] = 0

    # flair_remove_margin[flair_remove_margin > 0] = 4

    flair_remove_symmetry = np.copy(flair_remove_margin)
    '''

    cuiboid_binary, wt_binary, interest_slice_symmetry_3rd, interest_slice_symmetry_2nd, interest_slice_symmetry_1st = detect_non_interest_slices_symmetry_v2(
        flair_temp_arr_sq_enhance)


    image_predict_soi = cuiboid_binary
    image_predict_soi[image_predict_soi != 0] = 4

    image_predict_roi = wt_binary
    image_predict_roi[image_predict_roi != 0] = 4

    '''
    output_path_soi = './Test_' + branch_name + '_SOI/'
    if not os.path.exists(output_path_soi):
        try:
            os.makedirs(output_path_soi)
        except:
            folder_exist = True
    '''

    output_path_roi = './Test_' + branch_name + '_ROI/'
    if not os.path.exists(output_path_roi):
        try:
            os.makedirs(output_path_roi)
        except:
            folder_exist = True

    # image_predict_soi = image_predict_soi.astype(np.int16)
    # new_image_predict_soi = nib.Nifti1Image(image_predict_soi, affine_array)
    # nib.save(new_image_predict_soi, os.path.join(output_path_soi + patient_ID + '.nii.gz'))

    image_predict_roi = image_predict_roi.astype(np.int16)
    new_image_predict_roi = nib.Nifti1Image(image_predict_roi, affine_array)
    nib.save(new_image_predict_roi, os.path.join(output_path_roi + patient_ID + '.nii.gz'))

    return np.sum(interest_slice_symmetry_3rd), np.sum(interest_slice_symmetry_2nd), np.sum(interest_slice_symmetry_1st)


start_time = datetime.datetime.now()
print('start time: ', start_time)

num_validation_patient = len(brats_validation_ID)
num_interest_slice_validation_3rd = np.zeros(num_validation_patient, dtype=np.int32)
num_interest_slice_validation_2nd = np.zeros(num_validation_patient, dtype=np.int32)
num_interest_slice_validation_1st = np.zeros(num_validation_patient, dtype=np.int32)
# print('patient ID,  number of interest slices, is_low_contrast')

####################################
#### multi_thresholds
#######################################

def Multi_Threads_Testing_Cases_in_ValidationSet(index_thread, validation_path, all_patient_ID):
    div_multi_th, mod_multi_th = divmod(len(all_patient_ID), num_threads)

    num_interest_slice_validation_3rd = np.zeros(div_multi_th, dtype=np.int32)
    num_interest_slice_validation_2nd = np.zeros(div_multi_th, dtype=np.int32)
    num_interest_slice_validation_1st = np.zeros(div_multi_th, dtype=np.int32)

    for j in range(div_multi_th):
        num_interest_slice_validation_3rd[j], num_interest_slice_validation_2nd[j], num_interest_slice_validation_1st[j] = Testing_Cases_in_ValidationSet(validation_path, all_patient_ID[index_thread * div_multi_th + j])

    return num_interest_slice_validation_3rd, num_interest_slice_validation_2nd, num_interest_slice_validation_1st


div_multi_th, mod_multi_th = divmod(num_validation_patient, num_threads)
ths = []

for j in range(num_threads):
    th = Multi_Threads.MyThread(Multi_Threads_Testing_Cases_in_ValidationSet,
                                args=(j, validation_path, brats_validation_ID))
    th.start()
    ths.append(th)
for th in ths:
    th.join()
for jj in range(num_threads):
    idx_start = jj * div_multi_th
    idx_end = (jj + 1) * div_multi_th
    num_interest_slice_validation_3rd[idx_start:idx_end], num_interest_slice_validation_2nd[idx_start:idx_end], num_interest_slice_validation_1st[idx_start:idx_end] = ths[jj].get_result()

del ths

if mod_multi_th != 0:
    ths = []
    for k in range(mod_multi_th):
        th = Multi_Threads.MyThread(Testing_Cases_in_ValidationSet, args=(validation_path, brats_validation_ID[div_multi_th * num_threads + k]))
        th.start()
        ths.append(th)
    for th in ths:
        th.join()
    for kk in range(mod_multi_th):
        idx_task = div_multi_th * num_threads + kk
        num_interest_slice_validation_3rd[idx_task], num_interest_slice_validation_2nd[idx_task], num_interest_slice_validation_1st[idx_task] = ths[kk].get_result()

    del ths
####


####

'''
for j in range(num_validation_patient):
    num_interest_slice_validation_3rd[j], num_interest_slice_validation_2nd[j], num_interest_slice_validation_1st[j], is_low_contrast = brats_test_post(validation_path, brats_validation_ID[j])
    print(brats_validation_ID[j], '  ', num_interest_slice_validation_3rd[j], '  ', num_interest_slice_validation_2nd[j], '  ', num_interest_slice_validation_1st[j], '  ', is_low_contrast)
'''

'''
for j in range(num_validation_patient):
    print(brats_validation_ID[j], '  ', num_interest_slice_validation_3rd[j], '  ',
          num_interest_slice_validation_2nd[j], '  ', num_interest_slice_validation_1st[j])

print('removal 3rd:', 1 - np.sum(num_interest_slice_validation_3rd)/num_validation_patient/img_D)
print('removal 2nd:', 1 - np.sum(num_interest_slice_validation_2nd)/num_validation_patient/img_W)
print('removal 1st:', 1 - np.sum(num_interest_slice_validation_1st)/num_validation_patient/img_H)
'''

end_time = datetime.datetime.now()

print('end time: ', end_time)
print('duraiton: %.0f seconds for %.0f patient cases, average %.4f seconds per case' % ((end_time - start_time).seconds, num_validation_patient, int((end_time - start_time).seconds) / num_validation_patient))
###############################################################
# test training-set post
##############################################################

def calculate_score_v5(predict_image, seg_image):
    predict_image = predict_image.astype(np.float64)
    seg_image = seg_image.astype(np.float64)

    shape_image = np.shape(seg_image)

    ###############
    predict_ET = np.copy(predict_image)
    predict_ET[predict_ET < 4] = 0
    predict_ET[predict_ET == 4] = 1
    seg_ET = np.copy(seg_image)
    seg_ET[seg_ET < 4] = 0
    seg_ET[seg_ET == 4] = 1

    predict_WT = np.copy(predict_image)
    predict_WT[predict_WT > 0] = 1
    seg_WT = np.copy(seg_image)
    seg_WT[seg_WT > 0] = 1

    predict_TC = np.copy(predict_image)
    predict_TC[predict_TC == 2] = 0
    predict_TC[predict_TC > 0] = 1
    seg_TC = np.copy(seg_image)
    seg_TC[seg_TC == 2] = 0
    seg_TC[seg_TC > 0] = 1

    score = np.zeros((1, 6), dtype=np.float64)

    ep = 0.0000001
    ################
    if np.sum(seg_ET) == 0 and np.sum(predict_ET) == 0:
        score[0, 0] = 1.0
        score[0, 3] = 1.0
    elif np.sum(seg_ET) == 0 and np.sum(predict_ET) > 0:
        score[0, 0] = 0.0
        score[0, 3] = np.nan
    else:
        score[0, 0] = 2 * np.sum(seg_ET * predict_ET) / (np.sum(seg_ET) + np.sum(predict_ET) + ep)
        score[0, 3] = np.sum(seg_ET * predict_ET) / (np.sum(seg_ET) + ep)

    ###########
    if np.sum(seg_WT) == 0 and np.sum(predict_WT) == 0:
        score[0, 1] = 1.0
        score[0, 4] = 1.0
    elif np.sum(seg_WT) == 0 and np.sum(predict_WT) > 0:
        score[0, 1] = 0.0
        score[0, 4] = np.nan
    else:
        score[0, 1] = 2 * np.sum(seg_WT * predict_WT) / (np.sum(seg_WT) + np.sum(predict_WT) + ep)
        score[0, 4] = np.sum(seg_WT * predict_WT) / (np.sum(seg_WT) + ep)

    #################
    if np.sum(seg_TC) == 0 and np.sum(predict_TC) == 0:
        score[0, 2] = 1.0
        score[0, 5] = 1.0
    elif np.sum(seg_TC) == 0 and np.sum(predict_TC) > 0:
        score[0, 2] = 0.0
        score[0, 5] = np.nan
    else:
        score[0, 2] = 2 * np.sum(seg_TC * predict_TC) / (np.sum(seg_TC) + np.sum(predict_TC) + ep)
        score[0, 5] = np.sum(seg_TC * predict_TC) / (np.sum(seg_TC) + ep)

    return score


def Testing_Cases_in_TrainingSet(input_dir_path, patient_path):
    if patient_path[3] == '/':
        patient_path2 = patient_path[4:len(patient_path)]
    else:
        patient_path2 = patient_path

    path_flair = os.path.join(input_dir_path + patient_path + '/' + patient_path2 + '_flair.nii.gz')
    # path_t1 = os.path.join(input_dir_path + patient_path + '/' + patient_path2 + '_t1.nii.gz')
    # path_t1ce = os.path.join(input_dir_path + patient_path + '/' + patient_path2 + '_t1ce.nii.gz')
    # path_t2 = os.path.join(input_dir_path + patient_path + '/' + patient_path2 + '_t2.nii.gz')
    path_seg = os.path.join(input_dir_path + patient_path + '/' + patient_path2 + '_seg.nii.gz')

    flair_temp = nib.load(path_flair)
    flair_temp_arr = flair_temp.get_fdata()
    flair_temp_arr_sq = np.squeeze(flair_temp_arr)

    '''
    flair_soi_cube = np.copy(flair_temp_arr_sq)
    flair_out = np.copy(flair_temp_arr_sq)
    
    t1_temp = nib.load(path_t1)
    t1_temp_arr = t1_temp.get_fdata()
    t1_temp_arr_sq = np.squeeze(t1_temp_arr)

    t1ce_temp = nib.load(path_t1ce)
    t1ce_temp_arr = t1ce_temp.get_fdata()
    t1ce_temp_arr_sq = np.squeeze(t1ce_temp_arr)

    t2_temp = nib.load(path_t2)
    t2_temp_arr = t2_temp.get_fdata()
    t2_temp_arr_sq = np.squeeze(t2_temp_arr)
    '''

    seg_temp = nib.load(path_seg)
    seg_temp_arr = seg_temp.get_fdata()
    seg_temp_arr_sq = np.squeeze(seg_temp_arr)

    affine_array = flair_temp.affine

    ########################################################################################################
    flair_temp_arr_sq_enhance, is_low_contrast = intensity_enhancing_v3(flair_temp_arr_sq, num_bin=256)

    #################################################################################################
    '''

    interest_slice_remove_margin_3rd, interest_slice_remove_margin_2nd, interest_slice_remove_margin_1st = remove_margin(flair_temp_arr_sq, t1_temp_arr_sq, t1ce_temp_arr_sq, t2_temp_arr_sq)

    flair_remove_margin = np.copy(flair_temp_arr_sq_enhance)

    for j in range(img_D):
        if interest_slice_remove_margin_3rd[j] == 0:
            flair_remove_margin[:, :, j] = 0

    for j in range(img_W):
        if interest_slice_remove_margin_2nd[j] == 0:
            flair_remove_margin[:, j, :] = 0

    for j in range(img_H):
        if interest_slice_remove_margin_1st[j] == 0:
            flair_remove_margin[j, :, :] = 0

    # flair_remove_margin[flair_remove_margin > 0] = 4

    flair_remove_symmetry = np.copy(flair_remove_margin)
    '''

    # cuiboid_binary, wt_binary, interest_slice_symmetry_3rd, interest_slice_symmetry_2nd, interest_slice_symmetry_1st, similarity_value = detect_non_interest_slices_symmetry_v2(flair_temp_arr_sq_enhance, seg_soi=seg_temp_arr_sq, patient_path=patient_path)
    cuiboid_binary, wt_binary, interest_slice_symmetry_3rd, interest_slice_symmetry_2nd, interest_slice_symmetry_1st = detect_non_interest_slices_symmetry_v2(flair_temp_arr_sq_enhance)

    image_predict_soi = cuiboid_binary
    image_predict_soi[image_predict_soi != 0] = 4

    image_predict_roi = wt_binary
    image_predict_roi[image_predict_roi != 0] = 4

    '''
    output_path_tts_soi = './TestTrainingSet_' + branch_name + '_SOI/'
    if not os.path.exists(output_path_tts_soi):
        try:
            os.makedirs(output_path_tts_soi)
        except:
            folder_exist = True
    '''

    output_path_tts_roi = './TestTrainingSet_' + branch_name + '_ROI/'
    if not os.path.exists(output_path_tts_roi):
        try:
            os.makedirs(output_path_tts_roi)
        except:
            folder_exist = True

    # image_predict_soi = image_predict_soi.astype(np.int16)
    # new_image_predict_soi = nib.Nifti1Image(image_predict_soi, affine_array)
    # nib.save(new_image_predict_soi, os.path.join(output_path_tts_soi + patient_path2 + '.nii.gz'))

    image_predict_roi = image_predict_roi.astype(np.int16)
    new_image_predict_roi = nib.Nifti1Image(image_predict_roi, affine_array)
    nib.save(new_image_predict_roi, os.path.join(output_path_tts_roi + patient_path2 + '.nii.gz'))

    # score_soi = calculate_score_v5(image_predict_soi, seg_temp_arr_sq)

    score_roi = calculate_score_v5(image_predict_roi, seg_temp_arr_sq)
    '''
    print(patient_path, '%.4f, %.4f, %.4f, %.4f, %.4f, %.4f' % (
        score_roi[0, 0], score_roi[0, 1], score_roi[0, 2], score_roi[0, 3], score_roi[0, 4], score_roi[0, 5]), is_low_contrast)
    '''

    return score_roi, np.sum(interest_slice_symmetry_3rd), np.sum(interest_slice_symmetry_2nd), np.sum(interest_slice_symmetry_1st)


print('########################################')
print('start testing patient cases in training set, and assess the processing quality')

num_patient_trainingset = len(brats_training_ID) #  np.min([total_patient, 335])

# score_soi = np.zeros((num_patient_trainingset, 6), dtype=np.float32)  #
score_roi = np.zeros((num_patient_trainingset, 6), dtype=np.float32)  #
interest_slice_3rd = np.zeros(num_patient_trainingset, dtype=np.int32)
interest_slice_2nd = np.zeros(num_patient_trainingset, dtype=np.int32)
interest_slice_1st = np.zeros(num_patient_trainingset, dtype=np.int32)
# similarity_value = np.zeros((num_patient_trainingset, 12), dtype=np.float32)

#####################################
#### multi_threshold
#####################################


def Multi_Threads_Testing_Cases_in_TrainingSet(index_thread, training_path, all_patient_ID):
    div_multi_th, mod_multi_th = divmod(len(all_patient_ID), num_threads)

    score_roi = np.zeros((div_multi_th, 6), dtype=np.float32)  #
    interest_slice_3rd = np.zeros(div_multi_th, dtype=np.int32)
    interest_slice_2nd = np.zeros(div_multi_th, dtype=np.int32)
    interest_slice_1st = np.zeros(div_multi_th, dtype=np.int32)

    for j in range(div_multi_th):
        score_roi[j:j+1, :], interest_slice_3rd[j], interest_slice_2nd[j], interest_slice_1st[j] = Testing_Cases_in_TrainingSet(training_path, all_patient_ID[index_thread * div_multi_th + j])

    return score_roi, interest_slice_3rd, interest_slice_2nd, interest_slice_1st


div_multi_th, mod_multi_th = divmod(num_patient_trainingset, num_threads)
ths = []

for j in range(num_threads):
    th = Multi_Threads.MyThread(Multi_Threads_Testing_Cases_in_TrainingSet,
                                args=(j, training_path, brats_training_ID))
    th.start()
    ths.append(th)
for th in ths:
    th.join()
for jj in range(num_threads):
    idx_start = jj * div_multi_th
    idx_end = (jj + 1) * div_multi_th
    score_roi[idx_start:idx_end, :], interest_slice_3rd[idx_start:idx_end], interest_slice_2nd[idx_start:idx_end], interest_slice_1st[idx_start:idx_end] = ths[jj].get_result()

del ths

if mod_multi_th != 0:
    ths = []
    for k in range(mod_multi_th):
        th = Multi_Threads.MyThread(Testing_Cases_in_TrainingSet, args=(training_path, brats_training_ID[div_multi_th * num_threads + k]))
        th.start()
        ths.append(th)
    for th in ths:
        th.join()
    for kk in range(mod_multi_th):
        idx_task = div_multi_th * num_threads + kk
        score_roi[idx_task:idx_task+1, :], interest_slice_3rd[idx_task], interest_slice_2nd[idx_task], interest_slice_1st[idx_task] = ths[kk].get_result()

    del ths
####

####

'''
for j in range(total_patient_test_trainingset):
    score_soi[j:(j+1), :], score_roi[j:(j+1), :], interest_slice_3rd[j], interest_slice_2nd[j], interest_slice_1st[j], is_low_contrast = brats_test_trainingset_post(training_path, brats_training_ID[j])
'''
# score_soi_mean = np.nanmean(score_soi, axis=0)
score_roi_mean = np.nanmean(score_roi, axis=0)
# similarity_value_mean = np.nanmean(similarity_value, axis=0)

# np.savetxt('./score_soi.txt', score_soi, fmt='%f')
# np.savetxt('./score_soi_mean.txt', score_soi_mean, fmt='%f')
# np.savetxt('./score_roi.txt', score_roi, fmt='%f')
# np.savetxt('./score_roi_mean.txt', score_roi_mean, fmt='%f')
# np.savetxt('./similarity_value.txt', similarity_value, fmt='%f')
# np.savetxt('./similarity_value_mean.txt', similarity_value_mean, fmt='%f')

# print('patient ID,  number of interest slices')
# for j in range(num_patient_trainingset):
#     print(brats_training_ID[j], '  ', interest_slice_3rd[j], '  ', interest_slice_2nd[j], '  ', interest_slice_1st[j])


print('scores of ROI')
print('ID, Dice_WT, Sensitivity_WT')

for j in range(num_patient_trainingset):
    print(brats_training_ID[j], '%.4f, %.4f' % (score_roi[j, 1], score_roi[j, 4]))

'''
print('removal 3rd:', 1 - np.sum(interest_slice_3rd)/num_patient_trainingset/img_D)
print('removal 2nd:', 1 - np.sum(interest_slice_2nd)/num_patient_trainingset/img_W)
print('removal 1st:', 1 - np.sum(interest_slice_1st)/num_patient_trainingset/img_H)

print('Mean scores (trainingset):')
print('mean scores of SOI')
print('Dice_ET, Dice_WT, Dice_TC, Sensitivity_ET, Sensitivity_WT, Sensitivity_TC')
print('%.4f, %.4f, %.4f, %.4f, %.4f, %.4f' % (
    score_soi_mean[0], score_soi_mean[1], score_soi_mean[2], score_soi_mean[3], score_soi_mean[4], score_soi_mean[5]))
'''

print('mean scores of ROI')
print('ID, Dice_WT, Sensitivity_WT')
print('%.4f, %.4f' % (score_roi_mean[1], score_roi_mean[4]))

'''
print('mean similarity:')
print('%.4f, %.4f, %.4f' % (
    similarity_value_mean[3], similarity_value_mean[7], similarity_value_mean[11]))
'''


Start testing: 
start time:  2023-12-08 17:46:41.763143
end time:  2023-12-08 17:49:15.680564
duraiton: 153 seconds for 219 patient cases, average 0.6986 seconds per case
########################################
start testing patient cases in training set, and assess the processing quality
scores of ROI
ID, Dice_WT, Sensitivity_WT
BraTS2021_01431 0.6556, 0.5007
BraTS2021_01439 0.1952, 0.8265
BraTS2021_01411 0.8383, 0.9470
BraTS2021_01428 0.5141, 0.3702
BraTS2021_01419 0.8194, 0.7160
BraTS2021_01420 0.7717, 0.9886
BraTS2021_01418 0.9171, 0.8743
BraTS2021_01434 0.8038, 0.9123
BraTS2021_01433 0.7217, 0.9615
BraTS2021_01425 0.8317, 0.7694
BraTS2021_01413 0.8955, 0.8708
BraTS2021_01415 0.6902, 0.5828
BraTS2021_01417 0.8280, 0.8106
BraTS2021_01416 0.8736, 0.8121
BraTS2021_01414 0.9204, 0.8626
BraTS2021_01430 0.7859, 0.7427
BraTS2021_01424 0.8246, 0.7330
BraTS2021_01436 0.7326, 0.9403
BraTS2021_01422 0.8937, 0.8569
BraTS2021_01423 0.8494, 0.9713
BraTS2021_01426 0.7769, 0.7468
BraTS2021_01438 

"\nprint('mean similarity:')\nprint('%.4f, %.4f, %.4f' % (\n    similarity_value_mean[3], similarity_value_mean[7], similarity_value_mean[11]))\n"